In [27]:
import numpy as np
import pickle
import os
import tensorflow as tf

C:\Users\Chankoo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
D = 5
K = 6
N = 10

In [4]:
ratings = [5,5,4,3,0]

In [5]:
W = {}

In [6]:
for k in range(K):
    W['w_'+str(k)] = np.random.randint(0,high = 5,size=(D,N) ,dtype='int16')

In [7]:
W2 = np.zeros((D,K,N),dtype='int16')

In [8]:
for k in range(K):
    for d in range(D): 
        W2[d][k] = W['w_'+str(k)][d]

In [9]:
W2.shape

(5, 6, 10)

In [330]:
mu_0 = np.array(np.random.rand(K) * 2.0 - np.ones((K)),dtype='float32')
mu_0

array([-0.70752466,  0.03806972,  0.02335534, -0.07184985, -0.21061131,
       -0.31549415], dtype=float32)

In [14]:
sigma_0 = np.array(np.identity(K,dtype='float32'))
sigma_0

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [104]:
alpha_hat_0 = np.random.normal(loc = mu_0, scale= sigma_0.diagonal(),size = (D,K) )
alpha_hat_0

array([[-0.24938298,  0.88092023,  0.96387932,  1.19321081, -0.1043185 ,
         1.03167607],
       [-1.36069066,  1.27536926,  0.25290939,  0.51437761, -0.013455  ,
        -0.91282985],
       [-0.95626935, -0.10907779, -0.26746498,  1.16686273, -0.99313553,
         0.2258339 ],
       [ 2.29420473, -1.32935291, -0.41357428,  1.27115083,  0.6426769 ,
         0.54380792],
       [-0.44568435, -0.39624808, -0.71660223,  1.67179923, -1.01756768,
        -0.7561521 ]])

In [110]:
alpha_0 = np.exp(alpha_hat_0)/np.sum(np.exp(alpha_hat_0))
alpha_0

array([[0.0144656 , 0.04479412, 0.0486687 , 0.06121348, 0.01672389,
        0.0520827 ],
       [0.00476104, 0.06645506, 0.02390448, 0.031048  , 0.01831466,
        0.00745084],
       [0.00713411, 0.01664448, 0.01420639, 0.05962169, 0.00687589,
        0.02326594],
       [0.18407834, 0.0049126 , 0.01227522, 0.06617532, 0.03529826,
        0.03197533],
       [0.01188732, 0.01248975, 0.00906621, 0.098786  , 0.00670993,
        0.00871464]])

In [202]:
beta_0

array([[-0.02548366,  0.08036695, -0.08206261,  0.08121232,  0.06949994,
         0.09651957, -0.05386621,  0.00248668,  0.05271601, -0.00615398],
       [-0.00534419,  0.07079478, -0.06157328,  0.0640165 , -0.05443381,
         0.01562076, -0.02882842, -0.09236123,  0.03597522, -0.05940575],
       [-0.03133282, -0.0516488 , -0.06427151,  0.03898093, -0.02691578,
        -0.0241933 , -0.08954012, -0.08698585,  0.02425431,  0.06478388],
       [ 0.00623898, -0.05383178, -0.08017184, -0.02157001,  0.06635117,
        -0.08926516, -0.03293576,  0.08168465,  0.09177446,  0.09616955],
       [-0.09591533,  0.02302901,  0.08371054,  0.07855335, -0.01681843,
         0.06564218, -0.05953276,  0.06019444, -0.0639237 , -0.04285208],
       [-0.05071892,  0.0355148 , -0.07885524, -0.02176891, -0.03722655,
         0.06862206,  0.09413073,  0.01144626,  0.04068504,  0.01559115]])

In [203]:
S_0 =  model.aspect_rating_T[0]
S_0

array([[0.86911262, 2.51133475, 0.79262366, 0.83292179, 1.3835091 ,
        1.72245512],
       [1.56970731, 1.28077938, 0.53736761, 1.05700677, 1.67815938,
        1.12308689],
       [0.85739531, 2.42023864, 0.69253216, 1.61605403, 1.38279465,
        1.37056831],
       [1.72155581, 3.5254284 , 0.53955592, 1.33774667, 1.32116659,
        1.49361631],
       [1.23771014, 2.17791606, 0.80838737, 1.46112601, 1.41067701,
        1.03103525]])

In [355]:
ratings = [5, 5, 4, 3, 0]

In [246]:
delta_0 = 1.0

In [206]:
sigma_inv_0 = np.linalg.inv(np.identity(K))

In [351]:
mu_data = mu_0
sigma_data = sigma_0
sigma_inv_data = np.linalg.inv(sigma_0)
delta_data = delta_0
s_data = S_0

rating_data = [5,5,4,3,0]
 
alpha_hat_data = alpha_hat_0

alpha_hat = tf.Variable(alpha_hat_0, name = 'aspect_weight_hat',dtype=tf.float32)
# mu = tf.placeholder(tf.float32,shape=(K),name='mu')
# sigma_inv = tf.placeholder(tf.float32,shape=(K,K),name='sigma_inv')
# delta = tf.placeholder(tf.float32,name='delta')
# s = tf.placeholder(tf.float32,shape=(D,K),name = 'aspect_rating')
# preceding = tf.placeholder(tf.float32,shape=(D,K),name='precending')

mu = tf.placeholder_with_default(mu_data,shape=(K),name='mu')
sigma_inv = tf.placeholder_with_default(sigma_inv_data,shape=(K,K),name='sigma_inv')
delta = tf.placeholder_with_default(delta_data,shape=None,name='delta')
s = tf.placeholder_with_default(s_data,shape=(D,K),name = 'aspect_rating')




# rating = tf.placeholder(tf.int16,shape=(D))
 
    
preceding_data = np.zeros((D,K),dtype='float32')
for d in range(D):
    preceding_data[d] = (((rating_data - np.matmul(alpha_hat_data,s_data.T).diagonal())[d]*s_data[d]) )

preceding = tf.placeholder_with_default(preceding_data,shape=(D,K),name='precending')
loss =  preceding/tf.square(delta) + tf.matmul((alpha_hat-mu),sigma_inv)


optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(201):
#         sess.run([alpha_hat,train] # feed_dict 넣으면 에러 왜지
#                  , feed_dict={preceding: preceding_data,delta:delta_data,alpha_hat:alpha_hat_data,sigma_inv:sigma_inv_data,mu:mu_data})
        res = sess.run([alpha_hat,train])
        if step % 20 == 0:
            print(step, sess.run(alpha_hat))


0 [[-0.25938296  0.87092024  0.95387936  1.1832108  -0.1143185   1.0216761 ]
 [-1.3706907   1.2653693   0.24290939  0.5043776  -0.023455   -0.92282987]
 [-0.9662693  -0.11907779 -0.277465    1.1568627  -1.0031356   0.2158339 ]
 [ 2.2842047  -1.3393528  -0.42357427  1.2611508   0.6326769   0.53380793]
 [-0.45568433 -0.40624806 -0.7266022   1.6617992  -1.0275676  -0.7661521 ]]
20 [[-0.45938277  0.67092043  0.75387955  0.98321104 -0.3143185   0.82167625]
 [-1.5706905   1.0653695   0.04290935  0.3043778  -0.22345503 -1.1228297 ]
 [-1.1662692  -0.3190778  -0.4774648   0.9568629  -1.2031354   0.01583387]
 [ 2.084205   -1.5393527  -0.6235741   1.061151    0.4326771   0.33380812]
 [-0.6556842  -0.6062479  -0.926602    1.4617994  -1.2275674  -0.9661519 ]]
40 [[-0.6593826   0.47092062  0.55387974  0.78321123 -0.51431835  0.62167645]
 [-1.7706903   0.8653697  -0.15709065  0.10437778 -0.42345488 -1.3228295 ]
 [-1.366269   -0.5190776  -0.6774646   0.7568631  -1.4031352  -0.18416613]
 [ 1.8842051  -

In [348]:
S_0.shape

(5, 6)

In [612]:
class LRR_Model:
    global W2
    W = W2
    SCORE_SQUARE = False
    
    K = None # num of aspects
    D = None # num of documents
    N = None # num of words
    
    T = 1000 # max iteration
    
    mu_T = [0 for _ in range(T)]
    sigma_T = [0 for _ in range(T)]
    sigma_inv_T = [0 for _ in range(T)]
    delta_T = [0 for _ in range(T)]
    beta_T = [0 for _ in range(T)]
    
    aspect_weight_T = [0 for _ in range(T)] # alpha
    alpha_hat_T = [0 for _ in range(T)]
    
    aspect_rating_T = [0 for _ in range(T)] # S
    
    ratings = None
    
    def __init__(self,k,d,n,ratings:list):
        self.K = k
        self.D = d
        self.N = n
        self.ratings = np.array(ratings,dtype='float64')
        
        self.mu_T[0] = np.array(np.random.rand(self.K) * 2.0 - np.ones(self.K) ,dtype='float64')
        self.sigma_T[0] = np.array(np.identity(self.K),dtype='float64')
        self.sigma_inv_T[0] = np.linalg.inv(self.sigma_T[0])
        
        self.beta_T[0] = (2*np.random.rand(self.K, self.N)-1.0)/10
        self.delta_T[0] = np.ones(1,dtype='float64')
        
        self.alpha_hat_T[0] = np.random.normal(loc = self.mu_T[0], scale= self.sigma_T[0].diagonal(),size=(D,K) )
        self.aspect_weight_T[0] = np.exp(self.alpha_hat_T[0])/np.sum(np.exp(self.alpha_hat_T[0]))
        
    
#     def calc_covariance(self,vct):
#         '''
#         :param vct:
#         :return:

#         double sum = 0, s;
# 		for(int i=0; i<m_k; i++){
# 			s = 0;
# 			for(int j=0; j<m_k; j++)
# 				s += vct[j] * m_sigma_inv[j][i];
# 			sum += s * vct[i];
# 		}
# 		return sum;

#         '''

#     def calc_det(self):
#         return np.linalg.det(self.sigma)

#     def calc_sigma_inv(self,scale):
#         self.sigma_inv = np.linalg.inv(self.sigma) * scale

    def save(self,file_name:str):
        print(os.path.join(os.getcwd(), os.path.join('model', file_name)))
        with open(os.path.join(os.getcwd(), os.path.join('model', file_name)), 'wb') as fp:
            pickle.dump(self,fp)

    @classmethod
    def load(cls,file_name:str):
        try:
            with open(os.path.join(os.getcwd(),os.path.join('model',file_name)),'rb') as fp:
                return pickle.load(fp)
        except Exception as e:
            print(e)
    
    def fill_aspect_rating(self,t):
        aspect_rating = np.zeros((D,K))
        if self.SCORE_SQUARE==True:
            for d in range(D):
                aspect_rating[d] = 0.5*np.sum(np.multiply(self.beta_T[t],self.W[d]),axis=1)#to avoid negative rating 
            self.aspect_rating_T[t] = aspect_rating
        else:
            for d in range(D):
                aspect_rating[d] = np.exp(np.sum(np.multiply(self.beta_T[t],self.W[d]),axis=1))#to avoid negative rating 
            self.aspect_rating_T[t] = aspect_rating
            
    def infer_aspect_weight(self):

        alpha_hat = tf.Variable(self.alpha_hat_T[0], name = 'aspect_weight_hat',dtype=tf.float64)

        mu = tf.placeholder_with_default(self.mu_T[0],shape=(K),name='mu')
        sigma_inv = tf.placeholder_with_default(self.sigma_inv_T[0],shape=(K,K),name='sigma_inv')
        delta = tf.placeholder_with_default(self.delta_T[0],shape=None,name='delta')
        s = tf.placeholder_with_default(self.aspect_rating_T[0],shape=(D,K),name = 'aspect_rating')
        ratings = tf.placeholder_with_default(self.ratings,shape=(D),name = 'ratings')

#         preceding_tmp = np.zeros((D,K),dtype='float32')
#         for d in range(D):
#             preceding_tmp[d] = (((self.ratings - np.matmul(self.alpha_hat_T[0],self.aspect_rating_T[0].T).diagonal())[d]*self.aspect_rating_T[0][d]) )

#         preceding = tf.placeholder_with_default(preceding_tmp,shape=(D,K),name='precending')
        
        loss = tf.square( ratings - tf.linalg.diag_part(tf.matmul(alpha_hat,tf.transpose(s)) ))/(2*tf.square(delta)) + 0.5*tf.matmul(tf.matmul(tf.subtract(alpha_hat,mu), sigma_inv),tf.transpose(tf.subtract(alpha_hat,mu)))

#         grad =  preceding/tf.square(delta) + tf.matmul((alpha_hat-mu),sigma_inv)

        optimizer = tf.train.GradientDescentOptimizer(0.001)
        train = optimizer.minimize(loss)
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())

            for step in range(201):
                res = sess.run([alpha_hat,train])
                if step % 20 == 0:
                    print(step,':')
                    print('loss:',sess.run(loss),'\n') 
                    print('alpha_hat:',sess.run(alpha_hat),'\n')
        print('result:',res[0])

    
    def e_step():
        #infer_aspect_weight()
        pass
    def m_step():
        pass
    
    def em_est(converge:float):
#         e_step()
#         m_step()
        pass

In [613]:
model = LRR_Model(6,5,10,ratings)

In [614]:
model.save('test.model')

C:\Users\Chankoo\Desktop\GitHub\BOAZ-projects\airbnb-NLP\model\test.model


In [615]:
model = LRR_Model.load('test.model')

In [616]:
model.fill_aspect_rating(0)

In [617]:
model.aspect_rating_T[0]

array([[0.39832176, 0.56760971, 0.80171447, 1.4554806 , 1.02222094,
        0.80951094],
       [0.75669864, 0.7599482 , 1.21590412, 1.10563484, 1.12902233,
        0.68276736],
       [0.60445405, 0.47784935, 0.99632811, 0.8517048 , 0.92164397,
        0.54869252],
       [0.57487901, 0.95427958, 0.42680908, 1.29230463, 0.91091262,
        1.36451713],
       [0.89017478, 0.59392093, 0.67035177, 1.72723245, 1.34795057,
        0.52122495]])

In [618]:
model.infer_aspect_weight() 

0 :
loss: [[ 7.26592326  1.09964463  3.16368401  8.04013833  0.24624355]
 [ 6.28793394  3.37423178  4.07109154 10.81907101  0.44611301]
 [ 5.46697817  1.18609639  7.55853608 10.06087574 -0.37024145]
 [ 5.83204693  3.42269029  5.54949018 11.94889954  0.08954825]
 [ 5.94724896  0.95882911  3.02746979  7.99864506  1.4211611 ]] 

alpha_hat: [[ 0.89418232  0.38143726  0.73231246 -0.13649226 -0.80850131  1.69898906]
 [ 1.42686552  1.95782716  1.49266372  1.29522187 -0.09259236  0.92702311]
 [ 2.20568328  0.89520648 -0.671558    2.15147759 -2.08430444  0.44588538]
 [ 2.37822997  1.65329435  0.84253069  2.0997198   0.11364831  0.6979372 ]
 [ 0.15993196  0.4231479   0.85315472  0.44798643 -0.51900485 -0.30258381]] 

20 :
loss: [[ 3.68229334  0.69925218  1.58197039  2.34895658 -0.27353646]
 [ 2.96512424  2.06532304  2.57907639  3.92308438 -0.05266966]
 [ 1.92360813  0.65484208  5.78285831  4.11783356 -0.91655098]
 [ 2.46234441  1.77060015  3.88958365  4.64803711 -0.17219185]
 [ 2.13682046  0.091